# Analysis of Point Totals By Team (and Map)

## For Spread Bets

In [37]:
import os, sys, subprocess, json, time
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
xkcd_colors = sns.xkcd_rgb
GOLLY_GRAY = "#272B30"

## Loading Match Data

Start by loading match data from the `data/` dir:

In [39]:
def run_cmd(cmd_str):
    child = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE)
    output = child.communicate()[0].decode()
    return output

print(run_cmd('ls ../data/'))

clone.sh
gollyx-dragon-data
gollyx-hellmouth-data
gollyx-klein-data
gollyx-pseudo-data
gollyx-rainbow-data
gollyx-star-data
gollyx-toroidal-data



## Method to load regular season data

user specifies the cup and the season.

In [40]:
def fetch_season_data(cup, which_season0):
    cup = cup.lower()
    seas_file = os.path.join('..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', 'season.json')
    if not os.path.exists(seas_file):
        raise Exception(f"Error: season {which_season0} not valid: {seas_file} does not exist")
    with open(seas_file, 'r') as f:
        season0_seas = json.load(f)
    return season0_seas

In [41]:
def fetch_postseason_data(cup, which_season0):
    cup = cup.lower()
    post_file = os.path.join('..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', 'postseason.json')
    if not os.path.exists(post_file):
        raise Exception(f"Error: season {which_season0} not valid: {post_file} does not exist")
    with open(post_file, 'r') as f:
        season0_post = json.load(f)
    return season0_post

## Test drive

We use the methods above to extract point totals and splits for each game, and collate it by map.

In [42]:
season0 = 3
sdat = fetch_season_data('klein', season0)
pdat = fetch_postseason_data('klein', season0)

print(len(sdat))

11


In [43]:
# # Print a sample game
# pprint(sdat[0][0])

# Get info about the map
# print(sdat[0][0]['map'].keys())
print(sdat[0][0]['map']['mapName'])

# Get the point total
print(sdat[0][0]['team1Score'] + sdat[0][0]['team2Score'])

# Get the differential
print(abs(sdat[0][0]['team1Score'] - sdat[0][0]['team2Score']))

East Hellmouth's Revenge
528
398


## Method to assemble point totals by team & map name

Compile methods to extract and compile point totals, given either a season and a postseason data structure.

Then iterate over each season number, request the data, and compile the map point totals.

In [44]:
def _handle_it(game_, totals_):
    map_name = game_['map']['mapName']

    tup1 = (map_name, game_['team1Abbr'])
    if tup1 not in totals_:
        totals_[tup1] = []

    tup2 = (map_name, game_['team2Abbr'])
    if tup2 not in totals_:
        totals_[tup2] = []

    totals_[tup1].append(game_['team1Score'])
    totals_[tup2].append(game_['team2Score'])

In [45]:
def compile_totals_diffs_by_map(season_dat, postseason_dat, totals={}):

    # Season first
    for day in season_dat:
        for game in day:
            _handle_it(game, totals)

    # Postseason second
    for series in postseason_dat:
        miniseason = postseason_dat[series]
        for day in miniseason:
            for game in day:
                _handle_it(game, totals)
    
    return totals

### Try it out on a single season

Here is what the above method returns when run on a single season:

In [46]:
season0 = 0
sdat = fetch_season_data('klein', season0)
pdat = fetch_postseason_data('klein', season0)
totals = compile_totals_diffs_by_map(sdat, pdat)
pprint(totals)

{('Because Math', 'BPT'): [434, 91],
 ('Because Math', 'DECO'): [275],
 ('Because Math', 'DET'): [334, 233, 102],
 ('Because Math', 'EA'): [174],
 ('Because Math', 'LBFB'): [452],
 ('Because Math', 'MILF'): [404, 577],
 ('Because Math', 'OSHA'): [202],
 ('Because Math', 'SAC'): [245],
 ('Because Math', 'SGE'): [384],
 ('Because Math', 'SLC'): [155, 467],
 ('Because Math', 'SS'): [309],
 ('Because Math', 'VV'): [341, 331],
 ('Carcinization', 'BPT'): [127, 163],
 ('Carcinization', 'DECO'): [109],
 ('Carcinization', 'EA'): [120],
 ('Carcinization', 'FF'): [391, 108],
 ('Carcinization', 'SDBA'): [132],
 ('Carcinization', 'SFBS'): [115],
 ('Carcinization', 'SLC'): [305],
 ('Carcinization', 'SS'): [391],
 ('Charlie Foxtrot', 'AA'): [195, 25],
 ('Charlie Foxtrot', 'DET'): [327, 382, 298, 437],
 ('Charlie Foxtrot', 'EA'): [165],
 ('Charlie Foxtrot', 'FFF'): [83],
 ('Charlie Foxtrot', 'MILF'): [197, 130],
 ('Charlie Foxtrot', 'SDBA'): [135, 154],
 ('Charlie Foxtrot', 'SGE'): [55],
 ('Charlie Fo

### Try it out on multiple seasons

If we don't pass in a totals/diffs counter dictionary, then the method will automatically create an empty dictionary to start with, and manage that for us.

But we can also pass in our own totals/diffs counter dictionary, one that is accumulating totals from across multiple seasons.

In [47]:
totals = {}
for season0 in range(0, 23):
    sdat = fetch_season_data('klein', season0)
    pdat = fetch_postseason_data('klein', season0)
    compile_totals_diffs_by_map(sdat, pdat, totals)
print(totals[('West Hellmouth', 'SLC')])

[203, 263, 45, 280, 139, 188, 389, 99, 188, 330, 21, 249, 159, 78, 104, 145, 315]


## Quantile Function

We can use the statsmodels package to construct an empirical CDF function that can be evaluated at specific x locations (number of points), and will return the cumulative probability of that number of points (i.e., probability that the point total will be less than or equal to that value).

In [51]:
all_keys = totals.keys()
all_maps = sorted(list(set([t[0] for t in all_keys])))
all_team_abbrs = sorted(list(set([t[1] for t in all_keys])))

keys = [
    ('Because Math', 'AA'),
    ('Because Math', 'BB'),
    ('Carcinization', 'AA'),
    ('Carcinization', 'BB'),
    ('Engine Room', 'SS'),
    ('Engine Room', 'SFBS'),
]

for k in keys:
    q = np.quantile(totals[k], q=0.50)
    print(f"50th quantile for points, for map {k[0]} for team {k[1]}: {q:.0f}")

50th quantile for points, for map Because Math for team AA: 262
50th quantile for points, for map Because Math for team BB: 306
50th quantile for points, for map Carcinization for team AA: 130
50th quantile for points, for map Carcinization for team BB: 170
50th quantile for points, for map Engine Room for team SS: 297
50th quantile for points, for map Engine Room for team SFBS: 334
